In [ ]:
!pip install google-cloud-aiplatform
!pip install utils

  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13905 sha256=915ba2415890f622ace94f0fcfbd234b879cb8da5f6aaeb19471d4289ef167e8
  Stored in directory: /root/.cache/pip/wheels/b8/39/f5/9d0ca31dba85773ececf0a7f5469f18810e1c8a8ed9da28ca7
Successfully built utils


In [ ]:
from google.colab import files

# JSON dosyasını yüklemek için
uploaded = files.upload()


Saving delta-guild-433103-d3-11edc32db667.json to delta-guild-433103-d3-11edc32db667.json


In [ ]:
import os
from google.auth import default

# JSON dosyasının yolunu ayarla
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/delta-guild-433103-d3-11edc32db667.json"

def authenticate():
    credentials, project_id = default()
    return credentials, project_id

# Credentials ve proje kimliğini al
credentials, PROJECT_ID = authenticate()
print("Authenticated Project ID:", PROJECT_ID)


Authenticated Project ID: delta-guild-433103-d3


In [ ]:
REGION = 'us-central1'

In [ ]:
# Import and initialize the Vertex AI Python SDK

import vertexai
vertexai.init(project = PROJECT_ID,
              location = REGION,
              credentials = credentials)

In [ ]:
from vertexai.language_models import TextEmbeddingModel

In [ ]:
embedding_model = TextEmbeddingModel.from_pretrained(
    "textembedding-gecko@001")

In [ ]:
embedding = embedding_model.get_embeddings(
    ["life"])

In [ ]:
vector = embedding[0].values
print(f"Length = {len(vector)}")
print(vector[:10])

Length = 768
[-0.006005102302879095, 0.015532972291111946, -0.030447669327259064, 0.05322219058871269, 0.014444807544350624, -0.0542873740196228, 0.045140113681554794, 0.02127358317375183, -0.06537645310163498, 0.019103270024061203]


In [ ]:
embedding = embedding_model.get_embeddings(
    ["What is the meaning of life?"])

In [ ]:
vector = embedding[0].values
print(f"Length = {len(vector)}")
print(vector[:10])

Length = 768
[0.020522113889455795, 0.02229207195341587, -0.009265718050301075, 0.005001612473279238, 0.016248879954218864, -0.018983161076903343, 0.04320966452360153, 0.02643178217113018, -0.04369377717375755, 0.023666976019740105]


# Similarity
Calculate the similarity between two sentences as a number between 0 and 1.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
emb_1 = embedding_model.get_embeddings(
    ["What is the meaning of life?"]) # 42!

emb_2 = embedding_model.get_embeddings(
    ["How does one spend their time well on Earth?"])

emb_3 = embedding_model.get_embeddings(
    ["Would you like a salad?"])

vec_1 = [emb_1[0].values]
vec_2 = [emb_2[0].values]
vec_3 = [emb_3[0].values]

Note: the reason we wrap the embeddings (a Python list) in another list is because the cosine_similarity function expects either a 2D numpy array or a list of lists.

In [ ]:
print(cosine_similarity(vec_1,vec_2))
print(cosine_similarity(vec_2,vec_3))
print(cosine_similarity(vec_1,vec_3))

[[0.65503744]]
[[0.52001556]]
[[0.54139322]]


From word to sentence embeddings
One possible way to calculate sentence embeddings from word embeddings is to take the average of the word embeddings.
This ignores word order and context, so two sentences with different meanings, but the same set of words will end up with the same sentence embedding.

In [ ]:
in_1 = "The kids play in the park."
in_2 = "The play was for kids in the park."

In [ ]:
#- Remove stop words like ["the", "in", "for", "an", "is"] and punctuation.
in_pp_1 = ["kids", "play", "park"]
in_pp_2 = ["play", "kids", "park"]

In [ ]:
# Generate one embedding for each word.  So this is a list of three lists.
embeddings_1 = [emb.values for emb in embedding_model.get_embeddings(in_pp_1)]

In [ ]:
#np.stack() fonksiyonu, birden fazla array'i yeni bir eksende birleştirir.
#Bu durumda, embeddings_1 listesindeki tüm embedding vektörleri bir araya getirilip iki boyutlu bir array (matris) oluşturulur.

import numpy as np
emb_array_1 = np.stack(embeddings_1)
print(emb_array_1.shape)

(3, 768)


In [ ]:
embeddings_2 = [emb.values for emb in embedding_model.get_embeddings(in_pp_2)]
emb_array_2 = np.stack(embeddings_2)
print(emb_array_2.shape)

(3, 768)


In [ ]:
#Take the average embedding across the 3 word embeddings
#We'll get a single embedding of length 768.

emb_1_mean = emb_array_1.mean(axis = 0)
print(emb_1_mean.shape)

emb_2_mean = emb_array_2.mean(axis = 0)

(768,)


In [ ]:
print(emb_1_mean[:4])
print(emb_2_mean[:4])

[-0.00385805 -0.00522636  0.00574341  0.03331106]
[-0.00385805 -0.00522636  0.00574341  0.03331106]


In [ ]:
print(in_1)
print(in_2)

The kids play in the park.
The play was for kids in the park.


In [ ]:
embedding_1 = embedding_model.get_embeddings([in_1])
embedding_2 = embedding_model.get_embeddings([in_2])

In [ ]:
vector_1 = embedding_1[0].values
print(vector_1[:4])
vector_2 = embedding_2[0].values
print(vector_2[:4])

[0.0039385221898555756, -0.020830577239394188, -0.002994248876348138, -0.007580515928566456]
[-0.01565515622496605, -0.012884826399385929, 0.01229254249483347, -0.0005865463172085583]


Görüldüğü üzere, direkt embeddinglerinin vektörlerini aldığımızda eşit çıkmadı. Ama kelimeleri ayırıp ortalamalarınıj alınca eşit çıktı.
Bunun nedeni kelimelerin pozisyonlarının ve bağlamının da önemli olması.